In [39]:
from __future__ import print_function
import os.path
import dalmatian as dm
import pandas as pd
import sys
from CCLE_postp_function import *
sys.path.insert(0, '../JKBio/')
import Datanalytics as da 
import TerraFunction as terra
%load_ext autoreload
%autoreload 2
%load_ext rpy2.ipython
from taigapy import TaigaClient
tc = TaigaClient()
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [3]:
namespace="broad-genomics-delivery"
workspace="Getz_IBM_CellLines_Exomes"
refnamespace="broad-firecloud-ccle"
refworkspace="DepMap_Mutation_Calling_CGA_pipeline"
source="ibm"
samplesetname = "19Q3interim"
release = samplesetname

In [4]:
wm = dm.WorkspaceManager(namespace, workspace)
refwm = dm.WorkspaceManager(refnamespace, refworkspace)

In [47]:
CreateDatasetWithNewCellLines(wm,refwm, source, samplesetname)

/home/jeremie/ccle_processing/CCLE_postp_function.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  samples = wfrom.get_samples()


uploading new samples
Successfully imported 1576 participants.
Updating many hound records. Switching to batch updates
Hound executing batch upload of 2 records
Successfully imported 1801 samples.
Updating many hound records. Switching to batch updates
Hound executing batch upload of 655566 records
creating a sample set
Successfully imported 1 sample sets:
  * 19Q3interim (1 samples)


# Run the pipeline

In [44]:
CGAmutations = refwm.get_config("ICE_CGA_Production_Analysis_Pipeline_Cell_Lines")
CGAmutations

{'deleted': False,
 'inputs': {'CGA_Production_Analysis_Workflow.CrossCheckLaneFingerprints_Task.HaplotypeDBForCrossCheck': 'workspace.HAP_DB_FOR_CC',
  'CGA_Production_Analysis_Workflow.gatk_acnv_only.one_thousand_genomes_common_variants_minor_allele_freq_five': 'workspace.COMMON_VARIANTS_1000_GENOMES',
  'CGA_Production_Analysis_Workflow.VEP_Task.GNOMAD_FILE_IDX': 'workspace.GNOMAD_IDX',
  'CGA_Production_Analysis_Workflow.lego_plotter_task.mut_categs': 'workspace.MUT_CATEGORIES',
  'CGA_Production_Analysis_Workflow.MutectFC_Task.mutectIntervals': 'workspace.MUTECT_FC_INTERVALS',
  'CGA_Production_Analysis_Workflow.hasPicardMetrics_normal': '',
  'CGA_Production_Analysis_Workflow.DB_SNP_VCF_IDX': 'workspace.DB_SNP_VCF_IDX',
  'CGA_Production_Analysis_Workflow.iceMAFPonFilter.PONFile': 'workspace.ICE_TOKEN_PON',
  'CGA_Production_Analysis_Workflow.ctrlName': '"CCLF_RCRF1009-Normal-SM-F3R8L"',
  'CGA_Production_Analysis_Workflow.targetIntervals': 'workspace.ILLUMINA_TARGET_INTERVALS',


In [ ]:
refwm.update_config(CGAmutations)
submission_id = refwm.create_submission(CGAmutations['name'], samplesetname,'sample_set',expression='this.samples')

In [ ]:
wait_for_submission(refwm, submission_id1)

In [5]:
filtercommonvar = refwm.get_config("common_variant_filter")
filtercommonvar

{'deleted': False,
 'inputs': {'CommonVariantFilter.maf': 'this.mutation_validator_validated_maf',
  'CommonVariantFilter.disable_whitelist': '',
  'CommonVariantFilter.docker_tag': '',
  'CommonVariantFilter.filter_noncoding': '',
  'CommonVariantFilter.min_filter_depth': '',
  'CommonVariantFilter.min_exac_ac': '',
  'CommonVariantFilter.preemptible': '',
  'CommonVariantFilter.sampleId': 'this.name',
  'CommonVariantFilter.SSD': '',
  'CommonVariantFilter.RAM': ''},
 'methodConfigVersion': 2,
 'methodRepoMethod': {'methodName': 'common_variant_filter',
  'methodVersion': 3,
  'methodNamespace': 'breardon',
  'methodUri': 'agora://breardon/common_variant_filter/3',
  'sourceRepo': 'agora'},
 'name': 'common_variant_filter',
 'namespace': 'breardon',
 'outputs': {'CommonVariantFilter.commonfilterTask.passCount': 'this.passCount_cga_cf',
  'CommonVariantFilter.commonfilterTask.passedMAF': 'this.passedMAF_cga_cf',
  'CommonVariantFilter.commonfilterTask.rejectCount': 'this.rejectCount_c

In [6]:
refwm.update_config(filtercommonvar)
submission_id = refwm.create_submission(filtercommonvar['name'], samplesetname, 'sample_set', expression='this.samples')

Successfully updated configuration breardon/common_variant_filter
Successfully created submission e40b47c5-ee8e-4b6c-93a5-88c17c09ae22.


In [7]:
wait_for_submission(refwm, submission_id)

NameError: name 'wait_for_submission' is not defined

In [5]:
filtermaf = refwm.get_config("filterMAF_on_CGA_pipeline")
filtermaf

{'deleted': False,
 'inputs': {'filterMAF_for_CGA_workflow.filterMaf.inMAFfn': 'this.passedMAF_cga_cf',
  'filterMAF_for_CGA_workflow.filterMaf.onlyCoding': '"true"',
  'filterMAF_for_CGA_workflow.filterMaf.minAltReads': '"2"',
  'filterMAF_for_CGA_workflow.filterMaf.minAF': '"0.1"',
  'filterMAF_for_CGA_workflow.filterMaf.maxExAC_AF_COSMIC': '"1E-4"',
  'filterMAF_for_CGA_workflow.filterMaf.minCoverage': '"4"',
  'filterMAF_for_CGA_workflow.filterMaf.COSMIChotspotMinCnt': '"10"',
  'filterMAF_for_CGA_workflow.filterMaf.intersectWithMafFN': '',
  'filterMAF_for_CGA_workflow.filterMaf.addExAC_column': '"gs://fc-secure-012d088c-f039-4d36-bde5-ee9b1b76b912/scripts/add_column.R"',
  'filterMAF_for_CGA_workflow.filterMaf.sample_id': 'this.name',
  'filterMAF_for_CGA_workflow.filterMaf.TCGAhotspotMinCnt': '"3"',
  'filterMAF_for_CGA_workflow.filterMaf.TCGAhotspotMutFN': 'workspace.TCGAhotspotMutFN',
  'filterMAF_for_CGA_workflow.filterMaf.maxPon_loglike': '"1"',
  'filterMAF_for_CGA_workflow

In [ ]:
refwm.update_config(filtermaf)
submission_id = refwm.create_submission(filtermaf['name'], samplesetname,'sample_set',expression='this.samples')

In [ ]:
wait_for_submission(refwm, submission_id1)

In [ ]:
updateAllSampleSet('CCLE_'+samplesetname, Allsample_setname='DM19Q2_PATHS_CORRECTED_V2')

In [6]:
aggregate = refwm.get_config("aggregateMAFs_selectFields")
aggregate

{'deleted': False,
 'inputs': {'aggregateMAFs_selectFields_workflow.aggregateMAFs_selectFields.outputFN_prfx': 'this.name',
  'aggregateMAFs_selectFields_workflow.aggregateMAFs_selectFields.num_preempt': '5',
  'aggregateMAFs_selectFields_workflow.aggregateMAFs_selectFields.disk_space': '1000',
  'aggregateMAFs_selectFields_workflow.aggregateMAFs_selectFields.aggregate_selectedFields_MAF_Script': '"gs://fc-ca01b420-53e8-4922-b152-8d5e9bdcc07f/scripts/aggregate_selectedFields_MAF.R"',
  'aggregateMAFs_selectFields_workflow.aggregateMAFs_selectFields.memory': '15',
  'aggregateMAFs_selectFields_workflow.aggregateMAFs_selectFields.inMafFNs': 'this.samples.outMAFfn',
  'aggregateMAFs_selectFields_workflow.aggregateMAFs_selectFields.SelectFields': '"Hugo_Symbol,Entrez_Gene_Id,Center,NCBI_Build,Chromosome,Start_position,End_position,Strand,Variant_Classification,Variant_Type,Reference_Allele,Tumor_Seq_Allele1,Tumor_Seq_Allele2,dbSNP_RS,dbSNP_Val_Status,Genome_Change,Annotation_Transcript,Tum

In [ ]:
refwm.update_config(aggregate)
submission_id = refwm.create_submission(aggregate['name'], samplesetname,'sample_set',expression='this.samples')

In [ ]:
wait_for_submission(refwm, submission_id1)

# Downloading from terra

In [ ]:
res = refwm.get_sample_sets().loc['all_samples']
res

In [5]:
res = refwm.get_sample_sets().loc['DM19Q2_COMPLETE_SET']
res = res['passedCGA_filteredMAF_aggregated']
! gsutil cp $res "temp/mutation.mergedmaf.txt"

Copying gs://fc-secure-012d088c-f039-4d36-bde5-ee9b1b76b912/aa8cbc26-e1f1-4100-b424-e2a461d6f10d/aggregateMAFs_selectFields_workflow/f25a70f4-56e3-4e3c-87f4-75ad36c090f4/call-aggregateMAFs_selectFields/DM19Q2_COMPLETE_SET.mergedMAF.txt...
- [1 files][ 92.3 MiB/ 92.3 MiB]                                                
Operation completed over 1 objects/92.3 MiB.                                     


# postprocessing

In [15]:
%%R
source('../JKBio/gkugener/RScripts/load_libraries_and_annotations.R')
load('../JKBio/gkugener/DM_OMICS/Annotations.RData') 
# There are some cell lines the celllinemapr does not know how to map so we need to load this data object for now (from old datasets)

2019-06-27 06:32:40::WARNING  R[write to console]: ── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──

2019-06-27 06:32:40::WARNING  R[write to console]: ✔ ggplot2 3.1.1     ✔ tidyr   0.8.3
✔ tibble  2.1.3     ✔ purrr   0.3.2
✔ ggplot2 3.1.1     ✔ forcats 0.4.0

2019-06-27 06:32:40::WARNING  R[write to console]: ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::arrange()    masks plyr::arrange()
✖ dplyr::collapse()   masks IRanges::collapse()
✖ dplyr::combine()    masks Biobase::combine(), BiocGenerics::combine()
✖ purrr::compact()    masks plyr::compact()
✖ dplyr::count()      masks plyr::count()
✖ dplyr::desc()       masks plyr::desc(), IRanges::desc()
✖ tidyr::expand()     masks S4Vectors::expand()
✖ dplyr::failwith()   masks plyr::failwith()
✖ dplyr::filter()     masks stats::filter()
✖ dplyr::first()      masks S4Vectors::first()
✖ dplyr::id()         masks plyr::id()
✖ dplyr::lag()        masks stats::lag()
✖ d

2019-06-27 06:32:43::WARNING  R[write to console]: Loading required package: graph

2019-06-27 06:32:43::WARNING  R[write to console]: 
Attaching package: ‘graph’


2019-06-27 06:32:43::WARNING  R[write to console]: The following object is masked from ‘package:XML’:

    addNode


2019-06-27 06:32:43::WARNING  R[write to console]: The following object is masked from ‘package:stringr’:

    boundary


2019-06-27 06:32:43::WARNING  R[write to console]: The following object is masked from ‘package:plyr’:

    join


2019-06-27 06:32:44::WARNING  R[write to console]: Loading required package: RCurl

2019-06-27 06:32:44::WARNING  R[write to console]: Loading required package: bitops

2019-06-27 06:32:44::WARNING  R[write to console]: 
Attaching package: ‘RCurl’


2019-06-27 06:32:44::WARNING  R[write to console]: The following object is masked from ‘package:tidyr’:

    complete


2019-06-27 06:32:44::WARNING  R[write to console]: Loading required package: httr

2019-06-27 06:32:44::WARNING

Fetching https://cds.team/taiga/api/dataset/msigdb-gene-set-collections-8453/2 
Status 200 


2019-06-27 06:32:44::WARNING  R[write to console]: No encoding supplied: defaulting to UTF-8.



loading cached data version from  ~/.taiga/msigdb-gene-set-collections-8453_2.toc 
Checking for cached file ~/.taiga/495ea37b8d744a428946a5628c5fe7ce_gsc-data.raw 


2019-06-27 06:32:45::WARNING  R[write to console]: Error in readChar(con, 5L, useBytes = TRUE) : cannot open the connection
Calls: <Anonymous> ... <Anonymous> -> <Anonymous> -> withVisible -> load -> readChar

2019-06-27 06:32:45::WARNING  R[write to console]: In addition: 
2019-06-27 06:32:45::WARNING  R[write to console]: Warning messages:

2019-06-27 06:32:45::WARNING  R[write to console]: 1: 
2019-06-27 06:32:45::WARNING  R[write to console]: In taiga2.resolve.id(data.id, data.name, data.version, data.dir,  :
2019-06-27 06:32:45::WARNING  R[write to console]: 
 
2019-06-27 06:32:45::WARNING  R[write to console]:  No filename passed, getting the first one by default

2019-06-27 06:32:45::WARNING  R[write to console]: 2: 
2019-06-27 06:32:45::WARNING  R[write to console]: In readChar(con, 5L, useBytes = TRUE) :
2019-06-27 06:32:45::WARNING  R[write to console]: 
 
2019-06-27 06:32:45::WARNING  R[write to console]:  cannot open compressed file '../JKBio/gkugener/DM_OMICS/Annotations.R


Error in readChar(con, 5L, useBytes = TRUE) : cannot open the connection
Calls: <Anonymous> ... <Anonymous> -> <Anonymous> -> withVisible -> load -> readChar


In [11]:
%%R
source('CCLE_postp_function.R')
release <- '19Q3interim'

2019-06-27 06:31:50::WARNING  R[write to console]: Loading required package: AnnotationDbi

2019-06-27 06:31:50::WARNING  R[write to console]: Loading required package: stats4

2019-06-27 06:31:50::WARNING  R[write to console]: Loading required package: BiocGenerics

2019-06-27 06:31:50::WARNING  R[write to console]: Loading required package: parallel

2019-06-27 06:31:50::WARNING  R[write to console]: 
Attaching package: ‘BiocGenerics’


2019-06-27 06:31:50::WARNING  R[write to console]: The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


2019-06-27 06:31:50::WARNING  R[write to console]: The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


2019-06-27 06:31:50::WARNING  R[write to console]: The following objects are masked from ‘package:base’:

    anyDuplicated, append,

In [16]:
%%R
library(tidyverse)
library(data.table)
library(magrittr)
library(taigr)
library(celllinemapr) # To pull out DepMap_IDs from CCLE_names where needed
newly_merged_maf <- 'temp/mutation.mergedmaf.txt'

In [17]:
%%R
newly_merged_maf <- readMutations(newly_merged_maf)

In [18]:
%%R
snps_indels <- createSNPs(newly_merged_maf)

### (1.5) Quick adding

Here, rather than rerunning the entire analysis, because we know we are adding only WES samples, we can download the previous release's MAF, add the samples, update any annotations, and perform any global filters at the end.

First we need to do an additional step of filtering on coverage and number 

In [19]:
%%R
previous.release.maf <- load.from.taiga(data.name='depmap-mutation-calls-9be3', data.version=12, data.file='depmap_19Q1_mutation_calls')
if (colnames(previous.release.maf)[1] == 'X1') {
 previous.release.maf[,1] <- NULL 
}

Fetching https://cds.team/taiga/api/dataset/depmap-mutation-calls-9be3/12 
Status 200 


2019-06-27 06:33:06::WARNING  R[write to console]: No encoding supplied: defaulting to UTF-8.



loading cached data version from  /home/jeremie/.taiga/depmap-mutation-calls-9be3_12.toc 


2019-06-27 06:33:06::WARNING  R[write to console]: Loading from cached file /home/jeremie/.taiga/8e592fa8c4fc4428a278918eb951c606_depmap-19q1-mutation-calls.rds



In [20]:
%%R
merged_latest_release <- addToMainMutation(previous.release.maf, snps_indels)

In [21]:
%%R
filtered <- filterAllelicFraction(merged_latest_release)

In [22]:
%%R
filtered <- filterMinCoverage(filtered$merged_latest_release, filtered$removed_from_maf)

In [23]:
%%R
clean_annotations <- mergeAnnotations(newly_merged_maf,previous.release.maf)

[1] "find conflicts..."
[1] "2.4126057267189 for Hugo_Symbol"
[1] "2.59379101594289 for Entrez_Gene_Id"
[1] "2.4949270606041 for Variant_Classification"
[1] "2.44204205671946 for Variant_Type"
[1] "2.40701411167781 for dbSNP_RS"
[1] "2.42017827431361 for dbSNP_Val_Status"
[1] "2.46702679395676 for Genome_Change"
[1] "2.50486087799072 for Annotation_Transcript"
[1] "2.44873420397441 for cDNA_Change"
[1] "2.47855456670125 for Codon_Change"
[1] "2.47216558059057 for Protein_Change"
[1] "2.52361447811127 for isDeleterious"
[1] "2.51363442738851 for isTCGAhotspot"
[1] "2.45796721378962 for TCGAhsCnt"
[1] "2.39245359500249 for isCOSMIChotspot"
[1] "2.46209506193797 for COSMIChsCnt"
[1] "2.46295743385951 for ExAC_AF"
[1] "combining..."
[1] "resolve conflicts.."


In [77]:
%%R
ready_for_upload <- addAnnotation(filtered$merged_latest_release, clean_annotations, colnames(previous.release.maf))

In [32]:
%%R
# Save the ready to upload file to upload to taiga
write.table(
  ready_for_upload, 
  paste0('temp/mutations.', release, '.tsv'), sep = '\t', quote = F, row.names = F)

# Validation

#### Compare to previous release

I would run some checks here comparing the results to the previous releases MAF. Namely:

- Count the total number of mutations per cell line, split by type (SNP, INS, DEL)
- Count the total number of mutations observed by position (group by chromosome, start position, end position and count the number of mutations)
- Look at specific differences between the two MAFs (join on DepMap_ID, Chromosome, Start position, End position, Variant_Type). I would do this for WES only

In [33]:
mutations = pd.read_csv('temp/mutations.'+release+'.tsv', sep='\t')

/home/jeremie/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (3,19,25,26,27,28,29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Remove less used 

In [34]:
len(mutations.columns)

32

In [35]:
mutations[mutations["Hugo_Symbol"]=="ACOT4"][mutations['Start_position']==74058831].values

/home/jeremie/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


array([['ACOT4', 122970, 37, '14', 74058831, 74058831, '+', 'Silent',
        'SNP', 'C', 'T', 'rs542718736', 'byFrequency',
        'g.chr14:74058831C>T', 'ENST00000326303.4', 'ACH-000047',
        'c.168C>T', 'c.(166-168)gcC>gcT', 'p.A56A', False, False, nan,
        False, 0.0, nan, '161:0', nan, nan, nan, nan, nan, nan],
       ['ACOT4', 122970, 37, '14', 74058831, 74058831, '+', 'Silent',
        'SNP', 'C', 'T', 'rs542718736', 'byFrequency',
        'g.chr14:74058831C>T', 'ENST00000326303.4', 'ACH-000954',
        'c.168C>T', 'c.(166-168)gcC>gcT', 'p.A56A', False, False, nan,
        False, 0.0, nan, '116:116', nan, nan, nan, nan, nan, nan],
       ['ACOT4', 122970, 37, '14', 74058831, 74058831, '+', 'Silent',
        'SNP', 'C', 'T', 'rs542718736', 'byFrequency',
        'g.chr14:74058831C>T', 'ENST00000326303.4', 'ACH-000993',
        'c.168C>T', 'c.(166-168)gcC>gcT', 'p.A56A', False, False, nan,
        False, 0.0, nan, '7:3', nan, nan, nan, nan, nan, nan],
       ['ACOT4', 12

In [49]:
ac_data = mutations[[val for val in mutations.columns.values if '_AC' in val]]

In [52]:
ac_names = ac_data.columns.values
ac_data = ac_data.values

In [67]:
ac_data.shape[0]

1250416

In [81]:
allnan

0

In [79]:
toofew = 0
allnan = 0
for pos, val in enumerate(ac_data):
    i = 0
    print(100*pos/ac_data.shape[0],end='\r')
    for p, v in enumerate(val):
        if v is np.nan:
            i+=1
    if i==7:
        mutations = mutations.drop[pos]
        allnan+=1

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.97303617356143886

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1.4827065552584125

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



62.178267072718246

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### basic counts

In [8]:
#Count the total number of mutations per cell line, split by type (SNP, INS, DEL)


,Hugo_Symbol,Entrez_Gene_Id,NCBI_Build,Chromosome,Start_position,End_position,Strand,Variant_Classification,Variant_Type,Reference_Allele,...,isCOSMIChotspot,COSMIChsCnt,ExAC_AF,CGA_WES_AC,SangerWES_AC,SangerRecalibWES_AC,RNAseq_AC,HC_AC,RD_AC,WGS_AC
0,VPS13D,55187,37,1,12359347,12359347,+,Nonsense_Mutation,SNP,C,...,False,0.0,NaN,NaN,34:221,34:213,NaN,NaN,NaN,NaN
1,AADACL4,343066,37,1,12726308,12726322,+,In_Frame_Del,DEL,CTGGCGTGACGCCAT,...,False,3.0,NaN,NaN,9:0,53:193,NaN,NaN,NaN,28:32
2,IFNLR1,163702,37,1,24484172,24484172,+,Silent,SNP,G,...,False,0.0,NaN,NaN,118:0,118:0,10:0,NaN,NaN,18:0
3,TMEM57,55219,37,1,25785018,25785019,+,Frame_Shift_Ins,INS,-,...,False,0.0,NaN,NaN,NaN,NaN,6:28,NaN,NaN,NaN
4,ZSCAN20,7579,37,1,33954141,33954141,+,Missense_Mutation,SNP,T,...,False,0.0,NaN,NaN,27:61,28:62,NaN,NaN,NaN,NaN
5,POU3F1,5453,37,1,38512139,38512139,+,Missense_Mutation,SNP,C,...,False,0.0,NaN,NaN,NaN,5:0,NaN,NaN,NaN,10:4
6,MAST2,23139,37,1,46498028,46498028,+,Silent,SNP,C,...,False,0.0,0.000008,NaN,NaN,NaN,23:63,33:70,NaN,8:22
7,GBP4,115361,37,1,89657103,89657103,+,Silent,SNP,G,...,False,0.0,0.000016,NaN,97:0,96:0,NaN,NaN,NaN,27:15
8,VAV3,10451,37,1,108247170,108247170,+,Splice_Site,SNP,A,...,False,0.0,NaN,NaN,11:32,11:30,NaN,NaN,NaN,15:18
9,NBPF20,100288142,37,1,148346689,148346689,+,Missense_Mutation,SNP,T,...,False,0.0,0.000017,NaN,257:206,241:196,6:34,NaN,NaN,21:42


In [ ]:
# Count the total number of mutations observed by position

Are mutation consistent?

In [ ]:
#  to check this, if you group all the mutations in the mutations table by Chromosome, Start_position, End_position, Reference_Allele, Tumor_Seq_Allele1 columns, they should all have the same annotation for the other columns (protein change, exac_af, etc...)


QC mutations, for a known dependency, check if it matches mutation of this gene. (if P53 is mutated, cannot have dependency on P53 or MDM2 MDM4/ inverse fir BRAF and KRAF to themselves)

# uploading on taiga

In [82]:
tc.update_dataset(dataset_permaname="depmap-mutations-maf-35fe",
                 upload_file_path_dict={'temp/mutations.'+release+'.tsv': 'TableTSV'},
                 dataset_description="Updating to "+release,
                 force_remove=True)

Uploading mutations.19Q3interim...
hitting https://cds.team/taiga/api/datafile/a917f2a858374d7fa58250039a18f620
Conversion and upload...:
	 Uploading to S3
	 Uploading to S3

	 Done: mutations.19Q3interim properly converted and uploaded
Creating the new version with these files:
	NEW: mutations.19Q3interim - TableTSV
hitting https://cds.team/taiga/api/datasetVersion

Dataset version with id 5e551f1932fa4844bf87c566793b1434 created. You can access to this dataset version directly with this url: https://cds.team/taiga/dataset_version/5e551f1932fa4844bf87c566793b1434


'5e551f1932fa4844bf87c566793b1434'